In [2]:
import six.moves.urllib as urllib
from collections import defaultdict
from io import StringIO
from PIL import Image
import numpy as np
import os
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2


"""
    This is needed since the notebook is stored in the real_object_detection folder on my computer.
    
"""
sys.path.append("C://Users//Aliyu Isah//Desktop//Internships//GRIP TASKS//Real-Object-Detection//models//real_object_detection")

from utils import label_map_util
from utils import visualization_utils as vis_util

In [3]:
"""
    The model to download.
    and 
    Path to frozen detection graph. This is the actual model that is used for the object detection
    and
    List of the strings that is used to add correct label for each box.
"""
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
 
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [4]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())

In [5]:
from object_detection.utils import ops as utils_ops
utils_ops.tf = tf.compat.v1
tf.gfile = tf.io.gfile
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
        

In [6]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [7]:
"""
    Initialize webcam or USB webcam to start streaming images
"""
cap = cv2.VideoCapture(0)

In [8]:
"""
    Using this to disable tensorflow version 2 warnings and behaviors
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()
"""
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        ret = True
        while (ret):
            ret,image_np = cap.read()
            
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            
            # Each box represents a part of the image where a particular object was detected.
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
   
            cv2.imshow('image',cv2.resize(image_np,(1280,960)))
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                cap.release()
                break

Instructions for updating:
non-resource variables are not supported in the long term
